In [1]:
import psycopg2 as pg
import pandas as pd

conn = pg.connect(host="localhost", port = 5433, database="postgres", user="postgres")

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table


game_results = create_pandas_table("""select game_id, away_team_id, home_team_id, away_score_ct, home_score_ct, game_dt, game_ct
         , extract(year from game_dt) as yr
         , (case when home_score_ct>away_score_ct then 1 else 0 end) as home_win
      from retrosheet_game
     where extract(year from game_dt)>=2019
     and away_team_id not in ('ALS', 'NLS')""")
game_results

,game_id,away_team_id,home_team_id,away_score_ct,home_score_ct,game_dt,game_ct,yr,home_win
0,HOU201910120,NYA,HOU,7,0,2019-10-12,0,2019.0,0
1,HOU201910130,NYA,HOU,2,3,2019-10-13,0,2019.0,1
2,NYA201910150,HOU,NYA,4,1,2019-10-15,0,2019.0,0
3,NYA201910170,HOU,NYA,8,3,2019-10-17,0,2019.0,0
4,NYA201910180,HOU,NYA,1,4,2019-10-18,0,2019.0,1
...,...,...,...,...,...,...,...,...,...
2461,WAS201909250,PHI,WAS,2,5,2019-09-25,0,2019.0,1
2462,WAS201909260,PHI,WAS,3,6,2019-09-26,0,2019.0,1
2463,WAS201909270,CLE,WAS,2,8,2019-09-27,0,2019.0,1
2464,WAS201909280,CLE,WAS,7,10,2019-09-28,0,2019.0,1


In [2]:
away_team_game_results=game_results.copy(deep=True).rename(columns={"away_team_id": "team_id", "home_team_id": "opp_team_id", "away_score_ct": "rs", "home_score_ct": "ra"})
away_team_game_results['win']=away_team_game_results['home_win'].apply(lambda x: 1-x)
home_team_game_results=game_results.copy(deep=True).rename(columns={"home_team_id": "team_id", "away_team_id": "opp_team_id", "home_score_ct": "rs", "away_score_ct": "ra", "home_win": "win"})
away_team_game_results

,game_id,team_id,opp_team_id,rs,ra,game_dt,game_ct,yr,home_win,win
0,HOU201910120,NYA,HOU,7,0,2019-10-12,0,2019.0,0,1
1,HOU201910130,NYA,HOU,2,3,2019-10-13,0,2019.0,1,0
2,NYA201910150,HOU,NYA,4,1,2019-10-15,0,2019.0,0,1
3,NYA201910170,HOU,NYA,8,3,2019-10-17,0,2019.0,0,1
4,NYA201910180,HOU,NYA,1,4,2019-10-18,0,2019.0,1,0
...,...,...,...,...,...,...,...,...,...,...
2461,WAS201909250,PHI,WAS,2,5,2019-09-25,0,2019.0,1,0
2462,WAS201909260,PHI,WAS,3,6,2019-09-26,0,2019.0,1,0
2463,WAS201909270,CLE,WAS,2,8,2019-09-27,0,2019.0,1,0
2464,WAS201909280,CLE,WAS,7,10,2019-09-28,0,2019.0,1,0


In [3]:
home_team_game_results

,game_id,opp_team_id,team_id,ra,rs,game_dt,game_ct,yr,win
0,HOU201910120,NYA,HOU,7,0,2019-10-12,0,2019.0,0
1,HOU201910130,NYA,HOU,2,3,2019-10-13,0,2019.0,1
2,NYA201910150,HOU,NYA,4,1,2019-10-15,0,2019.0,0
3,NYA201910170,HOU,NYA,8,3,2019-10-17,0,2019.0,0
4,NYA201910180,HOU,NYA,1,4,2019-10-18,0,2019.0,1
...,...,...,...,...,...,...,...,...,...
2461,WAS201909250,PHI,WAS,2,5,2019-09-25,0,2019.0,1
2462,WAS201909260,PHI,WAS,3,6,2019-09-26,0,2019.0,1
2463,WAS201909270,CLE,WAS,2,8,2019-09-27,0,2019.0,1
2464,WAS201909280,CLE,WAS,7,10,2019-09-28,0,2019.0,1


In [4]:
team_game_results=pd.concat([away_team_game_results, home_team_game_results], keys=['away', 'home'])
team_game_results

game_id team_id opp_team_id  rs  ra     game_dt  game_ct  \
away 0     HOU201910120     NYA         HOU   7   0  2019-10-12        0   
     1     HOU201910130     NYA         HOU   2   3  2019-10-13        0   
     2     NYA201910150     HOU         NYA   4   1  2019-10-15        0   
     3     NYA201910170     HOU         NYA   8   3  2019-10-17        0   
     4     NYA201910180     HOU         NYA   1   4  2019-10-18        0   
...                 ...     ...         ...  ..  ..         ...      ...   
home 2461  WAS201909250     WAS         PHI   5   2  2019-09-25        0   
     2462  WAS201909260     WAS         PHI   6   3  2019-09-26        0   
     2463  WAS201909270     WAS         CLE   8   2  2019-09-27        0   
     2464  WAS201909280     WAS         CLE  10   7  2019-09-28        0   
     2465  WAS201909290     WAS         CLE   8   2  2019-09-29        0   

               yr  home_win  win  
away 0     2019.0       0.0    1  
     1     2019.0       1.0    0  
     2     2019.0       0.0    1  
     3     2019.0       0.0    1  
     4     2019.0       1.0    0  
...           ...       ...  ...  
home 2461  2019.0       NaN    1  
     2462  2019.0       NaN    1  
     2463  2019.0       NaN    1  
     2464  2019.0       NaN    1  
     2465  2019.0       NaN    1  

[4932 rows x 10 columns]

In [5]:
team_game_results = team_game_results.assign(game_num=team_game_results.sort_values(['game_dt', 'game_ct']).groupby(['team_id']).cumcount()+1)
team_game_results = team_game_results.assign(running_win_total=team_game_results.sort_values(['game_num']).groupby(['team_id']).win.cumsum())
team_game_results = team_game_results.assign(wpct=team_game_results.running_win_total/team_game_results.game_num)
team_game_results['incoming_wins'] = team_game_results['running_win_total']-team_game_results['win']
team_game_results['incoming_wpct'] = team_game_results['incoming_wins']/(team_game_results['game_num']-1)
team_game_results[team_game_results['team_id']=='SFN'].sort_values(by='game_num')

game_id team_id opp_team_id  rs  ra     game_dt  game_ct  \
away 1818  SDN201903280     SFN         SDN   0   2  2019-03-28        0   
     1819  SDN201903290     SFN         SDN   1   4  2019-03-29        0   
     1820  SDN201903300     SFN         SDN   3   2  2019-03-30        0   
     1821  SDN201903310     SFN         SDN   1   3  2019-03-31        0   
     1093  LAN201904010     SFN         LAN   4   2  2019-04-01        0   
...                 ...     ...         ...  ..  ..         ...      ...   
home 2056  SFN201909250     SFN         COL   2   1  2019-09-25        0   
     2057  SFN201909260     SFN         COL   8   3  2019-09-26        0   
     2058  SFN201909270     SFN         LAN   2   9  2019-09-27        0   
     2059  SFN201909280     SFN         LAN   0   2  2019-09-28        0   
     2060  SFN201909290     SFN         LAN   0   9  2019-09-29        0   

               yr  home_win  win  game_num  running_win_total      wpct  \
away 1818  2019.0       1.0    0         1                  0  0.000000   
     1819  2019.0       1.0    0         2                  0  0.000000   
     1820  2019.0       0.0    1         3                  1  0.333333   
     1821  2019.0       1.0    0         4                  1  0.250000   
     1093  2019.0       0.0    1         5                  2  0.400000   
...           ...       ...  ...       ...                ...       ...   
home 2056  2019.0       NaN    1       158                 76  0.481013   
     2057  2019.0       NaN    1       159                 77  0.484277   
     2058  2019.0       NaN    0       160                 77  0.481250   
     2059  2019.0       NaN    0       161                 77  0.478261   
     2060  2019.0       NaN    0       162                 77  0.475309   

           incoming_wins  incoming_wpct  
away 1818              0            NaN  
     1819              0       0.000000  
     1820              0       0.000000  
     1821              1       0.333333  
     1093              1       0.250000  
...                  ...            ...  
home 2056             75       0.477707  
     2057             76       0.481013  
     2058             77       0.484277  
     2059             77       0.481250  
     2060             77       0.478261  

[162 rows x 15 columns]

In [6]:
prediction_factors=team_game_results[['game_id', 'team_id', 'incoming_wpct']]
prediction_factors

game_id team_id  incoming_wpct
away 0     HOU201910120     NYA       0.642424
     1     HOU201910130     NYA       0.644578
     2     NYA201910150     HOU       0.656805
     3     NYA201910170     HOU       0.658824
     4     NYA201910180     HOU       0.660819
...                 ...     ...            ...
home 2461  WAS201909250     WAS       0.560510
     2462  WAS201909260     WAS       0.563291
     2463  WAS201909270     WAS       0.566038
     2464  WAS201909280     WAS       0.568750
     2465  WAS201909290     WAS       0.571429

[4932 rows x 3 columns]

In [7]:
predictions = pd.merge(game_results, prediction_factors.rename(columns={"team_id": "away_team_id", "incoming_wpct": "away_incoming_wpct"}), how='inner', on=['game_id', 'away_team_id'])
predictions = pd.merge(predictions, prediction_factors.rename(columns={"team_id": "home_team_id", "incoming_wpct": "home_incoming_wpct"}), how='inner', on=['game_id', 'home_team_id'])
predictions['prediction']= predictions.apply(lambda gm: 1 if gm['home_incoming_wpct']>=gm['away_incoming_wpct'] else 0, axis=1)
predictions['prediction_correct']= (predictions['prediction']+predictions['home_win']+1)%2
predictions

,game_id,away_team_id,home_team_id,away_score_ct,home_score_ct,game_dt,game_ct,yr,home_win,away_incoming_wpct,home_incoming_wpct,prediction,prediction_correct
0,HOU201910120,NYA,HOU,7,0,2019-10-12,0,2019.0,0,0.642424,0.658683,1,0
1,HOU201910130,NYA,HOU,2,3,2019-10-13,0,2019.0,1,0.644578,0.654762,1,1
2,NYA201910150,HOU,NYA,4,1,2019-10-15,0,2019.0,0,0.656805,0.640719,0,1
3,NYA201910170,HOU,NYA,8,3,2019-10-17,0,2019.0,0,0.658824,0.636905,0,1
4,NYA201910180,HOU,NYA,1,4,2019-10-18,0,2019.0,1,0.660819,0.633136,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,WAS201909250,PHI,WAS,2,5,2019-09-25,0,2019.0,1,0.503185,0.560510,1,1
2462,WAS201909260,PHI,WAS,3,6,2019-09-26,0,2019.0,1,0.500000,0.563291,1,1
2463,WAS201909270,CLE,WAS,2,8,2019-09-27,0,2019.0,1,0.584906,0.566038,0,0
2464,WAS201909280,CLE,WAS,7,10,2019-09-28,0,2019.0,1,0.581250,0.568750,0,0


In [8]:
predictions.describe()

,away_score_ct,home_score_ct,game_ct,yr,home_win,away_incoming_wpct,home_incoming_wpct,prediction,prediction_correct
count,2466.000000,2466.000000,2466.000000,2466.0,2466.000000,2450.000000,2452.000000,2466.000000,2466.000000
mean,4.829278,4.807786,0.040146,2019.0,0.528386,0.500779,0.501569,0.509327,0.577859
std,3.419733,3.214627,0.255587,0.0,0.499295,0.116269,0.114478,0.500014,0.494001
min,0.000000,0.000000,0.000000,2019.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,0.000000,2019.0,0.000000,0.436336,0.436620,0.000000,0.000000
50%,4.000000,4.000000,0.000000,2019.0,1.000000,0.506370,0.508475,1.000000,1.000000
75%,7.000000,7.000000,0.000000,2019.0,1.000000,0.576923,0.571429,1.000000,1.000000
max,23.000000,21.000000,2.000000,2019.0,1.000000,1.000000,1.000000,1.000000,1.000000
